# SocialAL Model

### Fit single alpha with decay model to data - multiple subjects

KLS 3.3.22
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

In [2]:
import sys
print(sys.version)
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


In [3]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, alpha):
    Prob = Prob + alpha*(recip-Prob)
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    alpha = params[0]
    beta = params[1]
    decay = params[2]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #print(cprob)

            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], alpha)
            ev[trustee] = update_value(prob[trustee])
            
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(trustee)
        for x in options:
            prob[x] = prob[x] + decay * (0.5 - prob[x])
            ev[x] =  update_value(prob[x])       
    return totalLLH

def model_fit_once(data):
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [alpha, beta, decay]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (1e-10, 100), (0, 1)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

def model_fit(data):
    
    tries = 100 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [alpha, beta, decay]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (1e-10, 20), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
            
    return bestFit

Load and Clean Data

In [5]:
files = os.listdir('../../data/modeling/missing')

In [6]:
print(files)

['sub-2038.csv', 'sub-2039.csv', 'sub-2026.csv', 'sub-2027.csv', 'sub-2033.csv', 'sub-2025.csv', 'sub-2031.csv', 'sub-2030.csv', 'sub-2024.csv', 'sub-2034.csv', 'sub-2021.csv', 'sub-2037.csv', 'sub-2023.csv', 'sub-2022.csv', 'sub-2036.csv']


In [7]:
def load_and_clean(file):
    path = os.path.join('../../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    return(data)

Fit Model

In [8]:
print(getcontext())
params = [model_fit(load_and_clean(file)) for file in files]

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


In [9]:
params

[      fun: 61.42483725154438
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 2.74269496e-04, -1.35003109e-04, -6.75458974e-01])
   message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 312
       nit: 55
      njev: 78
    status: 0
   success: True
         x: array([6.00213871e-03, 2.00000000e+01, 1.00000000e+00]),
       fun: 34.35241933110275
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([0.00000000e+00, 7.10542732e-07, 2.84217094e-06])
   message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 68
       nit: 13
      njev: 17
    status: 0
   success: True
         x: array([0.83014351, 0.53184832, 0.02098698]),
       fun: 58.096880230264
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 2.45847787e-04, -3.62376765e-04, -6.39488459e-06])
   message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 304
       nit: 57
      njev: 76
    status: 0
   su

In [10]:
params = pd.DataFrame(params)
params['id'] = [file[:-4] for file in files]
params['alpha'] = [params['x'][y][0] for y in range(len(params))]
params['beta'] = [params['x'][y][1] for y in range(len(params))]
params['decay'] = [params['x'][y][2] for y in range(len(params))]
params['-LLH'] = [params['fun'][y] for y in range(len(params))]
params = params[['id', 'alpha', 'beta', 'decay', '-LLH']]
print(params)   

          id     alpha          beta     decay       -LLH
0   sub-2038  0.006002  2.000000e+01  1.000000  61.424837
1   sub-2039  0.830144  5.318483e-01  0.020987  34.352419
2   sub-2026  0.009392  2.000000e+01  0.532974  58.096880
3   sub-2027  0.127701  5.493601e-01  0.000000  47.877589
4   sub-2033  1.000000  1.183154e-01  1.000000  59.950593
5   sub-2025  0.002477  2.000000e+01  0.000000  50.730065
6   sub-2031  0.000000  1.000000e-10  1.000000  60.996952
7   sub-2030  0.000000  1.000000e-10  1.000000  58.224363
8   sub-2024  0.463954  4.272929e-01  0.000000  43.060880
9   sub-2034  0.001483  2.000000e+01  0.000000  53.163115
10  sub-2021  0.935232  1.868886e-01  0.000000  54.493332
11  sub-2037  1.000000  2.674687e-01  0.070692  49.838614
12  sub-2023  0.482881  6.157203e-01  0.000000  34.813597
13  sub-2022  0.536960  8.943738e-03  1.000000  60.995161
14  sub-2036  0.001895  2.000000e+01  1.000000  62.186124


In [11]:
# save parameters in text file
params.to_csv(path_or_buf = '../../output/missing_single_alpha_with_decay_model_params.csv', index = False)